# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 4 2022 8:30AM,243405,10,0085923027,ISDIN Corporation,Released
1,Aug 4 2022 8:30AM,243405,10,0085923023,ISDIN Corporation,Released
2,Aug 4 2022 8:30AM,243405,10,0085923024,ISDIN Corporation,Released
3,Aug 4 2022 8:30AM,243405,10,0085923029,ISDIN Corporation,Released
4,Aug 4 2022 8:30AM,243405,10,0085923028,ISDIN Corporation,Released
5,Aug 4 2022 8:30AM,243405,10,0085923031,ISDIN Corporation,Released
6,Aug 4 2022 8:30AM,243405,10,0085923026,ISDIN Corporation,Released
7,Aug 4 2022 8:30AM,243405,10,0085923025,ISDIN Corporation,Released
8,Aug 4 2022 8:30AM,243405,10,0085923042,ISDIN Corporation,Released
9,Aug 4 2022 8:30AM,243405,10,0085923058,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,243402,Released,3
41,243403,Released,13
42,243404,Released,1
43,243405,Released,11
44,243406,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243402,NaN,NaN,3.0
243403,NaN,NaN,13.0
243404,NaN,NaN,1.0
243405,NaN,NaN,11.0
243406,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243306,0.0,6.0,1.0
243316,0.0,0.0,1.0
243317,0.0,1.0,0.0
243318,0.0,1.0,0.0
243320,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243306,0,6,1
243316,0,0,1
243317,0,1,0
243318,0,1,0
243320,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243306,0,6,1
1,243316,0,0,1
2,243317,0,1,0
3,243318,0,1,0
4,243320,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243306,,6,1
1,243316,,,1
2,243317,,1,
3,243318,,1,
4,243320,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 4 2022 8:30AM,243405,10,ISDIN Corporation
11,Aug 3 2022 3:36PM,243406,12,Doctors Scientific Organica
12,Aug 3 2022 3:25PM,243404,12,Hush Hush
13,Aug 3 2022 3:24PM,243403,20,Alumier Labs Inc.
26,Aug 3 2022 2:50PM,243402,19,ACG North America LLC
29,Aug 3 2022 2:45PM,243401,10,Emerginnova
33,Aug 3 2022 2:44PM,243400,10,"ClearSpec, LLC"
34,Aug 3 2022 2:43PM,243399,10,"ClearSpec, LLC"
35,Aug 3 2022 2:40PM,243398,10,"Methapharm, Inc."
39,Aug 3 2022 2:40PM,243397,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 4 2022 8:30AM,243405,10,ISDIN Corporation,,,11
1,Aug 3 2022 3:36PM,243406,12,Doctors Scientific Organica,,,1
2,Aug 3 2022 3:25PM,243404,12,Hush Hush,,,1
3,Aug 3 2022 3:24PM,243403,20,Alumier Labs Inc.,,,13
4,Aug 3 2022 2:50PM,243402,19,ACG North America LLC,,,3
5,Aug 3 2022 2:45PM,243401,10,Emerginnova,,1,3
6,Aug 3 2022 2:44PM,243400,10,"ClearSpec, LLC",,,1
7,Aug 3 2022 2:43PM,243399,10,"ClearSpec, LLC",,,1
8,Aug 3 2022 2:40PM,243398,10,"Methapharm, Inc.",,,4
9,Aug 3 2022 2:40PM,243397,10,"Methapharm, Inc.",,,14


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 4 2022 8:30AM,243405,10,ISDIN Corporation,11,,
1,Aug 3 2022 3:36PM,243406,12,Doctors Scientific Organica,1,,
2,Aug 3 2022 3:25PM,243404,12,Hush Hush,1,,
3,Aug 3 2022 3:24PM,243403,20,Alumier Labs Inc.,13,,
4,Aug 3 2022 2:50PM,243402,19,ACG North America LLC,3,,
5,Aug 3 2022 2:45PM,243401,10,Emerginnova,3,1,
6,Aug 3 2022 2:44PM,243400,10,"ClearSpec, LLC",1,,
7,Aug 3 2022 2:43PM,243399,10,"ClearSpec, LLC",1,,
8,Aug 3 2022 2:40PM,243398,10,"Methapharm, Inc.",4,,
9,Aug 3 2022 2:40PM,243397,10,"Methapharm, Inc.",14,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 4 2022 8:30AM,243405,10,ISDIN Corporation,11,,
1,Aug 3 2022 3:36PM,243406,12,Doctors Scientific Organica,1,,
2,Aug 3 2022 3:25PM,243404,12,Hush Hush,1,,
3,Aug 3 2022 3:24PM,243403,20,Alumier Labs Inc.,13,,
4,Aug 3 2022 2:50PM,243402,19,ACG North America LLC,3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 4 2022 8:30AM,243405,10,ISDIN Corporation,11.0,NaN,NaN
1,Aug 3 2022 3:36PM,243406,12,Doctors Scientific Organica,1.0,NaN,NaN
2,Aug 3 2022 3:25PM,243404,12,Hush Hush,1.0,NaN,NaN
3,Aug 3 2022 3:24PM,243403,20,Alumier Labs Inc.,13.0,NaN,NaN
4,Aug 3 2022 2:50PM,243402,19,ACG North America LLC,3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 4 2022 8:30AM,243405,10,ISDIN Corporation,11.0,0.0,0.0
1,Aug 3 2022 3:36PM,243406,12,Doctors Scientific Organica,1.0,0.0,0.0
2,Aug 3 2022 3:25PM,243404,12,Hush Hush,1.0,0.0,0.0
3,Aug 3 2022 3:24PM,243403,20,Alumier Labs Inc.,13.0,0.0,0.0
4,Aug 3 2022 2:50PM,243402,19,ACG North America LLC,3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3894074,99.0,68.0,0.0
12,730172,3.0,0.0,0.0
15,243306,1.0,6.0,0.0
16,243354,0.0,1.0,0.0
19,973497,5.0,5.0,0.0
20,973544,21.0,7.0,7.0
21,1216598,3.0,2.0,0.0
22,243345,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3894074,99.0,68.0,0.0
1,12,730172,3.0,0.0,0.0
2,15,243306,1.0,6.0,0.0
3,16,243354,0.0,1.0,0.0
4,19,973497,5.0,5.0,0.0
5,20,973544,21.0,7.0,7.0
6,21,1216598,3.0,2.0,0.0
7,22,243345,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,99.0,68.0,0.0
1,12,3.0,0.0,0.0
2,15,1.0,6.0,0.0
3,16,0.0,1.0,0.0
4,19,5.0,5.0,0.0
5,20,21.0,7.0,7.0
6,21,3.0,2.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,99.0
1,12,Released,3.0
2,15,Released,1.0
3,16,Released,0.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
Executing,68.0,0.0,6.0,1.0,5.0,7.0,2.0,0.0
Released,99.0,3.0,1.0,0.0,5.0,21.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
1,Executing,68.0,0.0,6.0,1.0,5.0,7.0,2.0,0.0
2,Released,99.0,3.0,1.0,0.0,5.0,21.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
1,Executing,68.0,0.0,6.0,1.0,5.0,7.0,2.0,0.0
2,Released,99.0,3.0,1.0,0.0,5.0,21.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()